In [3]:
import pandas as pd

# Set display options to show more rows and columns
pd.set_option('display.max_columns', None) # Show all columns

listings = pd.read_csv('final_listings.csv')
#listings_without_amenities = pd.read_csv('final_listings_without_amenities.csv')

In [4]:
listings

,host_since,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,host_verification_email,host_verification_phone,host_verification_work_email,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in,sentiment,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,neighbourhood_group_cleansed_Eixample,neighbourhood_group_cleansed_Gràcia,neighbourhood_group_cleansed_Horta-Guinardó,neighbourhood_group_cleansed_Les Corts,neighbourhood_group_cleansed_Nou Barris,neighbourhood_group_cleansed_Sant Andreu,neighbourhood_group_cleansed_Sant Martí,neighbourhood_group_cleansed_Sants-Montjuïc,neighbourhood_group_cleansed_Sarrià-Sant Gervasi,property_type_Boat,property_type_Camper/RV,property_type_Casa particular,property_type_Earthen home,property_type_Entire cabin,property_type_Entire chalet,property_type_Entire condo,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home,property_type_Entire loft,property_type_Entire place,property_type_Entire rental unit,property_type_Entire serviced apartment,property_type_Entire townhouse,property_type_Entire vacation home,property_type_Entire villa,property_type_Private room,property_type_Private room in bed and breakfast,property_type_Private room in boat,property_type_Private room in casa particular,property_type_Private room in chalet,property_type_Private room in condo,property_type_Private room in cottage,property_type_Private room in dome,property_type_Private room in floor,property_type_Private room in guest suite,property_type_Private room in guesthouse,property_type_Private room in home,property_type_Private room in hostel,property_type_Private room in loft,property_type_Private room in rental unit,property_type_Private room in serviced apartment,property_type_Private room in tiny home,property_type_Private room in townhouse,property_type_Private room in vacation home,property_type_Private room in villa,property_type_Room in aparthotel,property_type_Room in bed and breakfast,property_type_Room in boutique hotel,property_type_Room in hostel,property_type_Room in hotel,property_type_Room in serviced apartment,property_type_Shared room in bed and breakfast,property_type_Shared room in boutique hotel,property_type_Shared room in condo,property_type_Shared room in floor,property_type_Shared room in guest suite,property_type_Shared room in guesthouse,property_type_Shared room in home,property_type_Shared room in hostel,property_type_Shared room in hotel,property_type_Shared room in loft,property_type_Shared room in rental unit,property_type_Shared room in serviced apartment,property_type_Shared room in villa,property_type_Tiny home,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,0.799129,1.000000,0.086502,0.002848,1.0,1.0,0.476053,0.636848,0.066667,0.020000,0.02,0.011923,0.135514,0.005343,0.253084,0.042433,0.014155,0.009804,0.935000,0.965124,0.886812,0.905000,0.958000,0.947361,0.891019,1.0,0.128569,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666727,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [5]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Separate features and target
X = listings.drop(columns=["price"])
#X_without_amenities = listings_without_amenities.drop(columns=["price"])

y = listings["price"]


# Set up 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)


def perform_cross_validation(X, y, model, kf):
    mse_scores = []
    mae_scores = []
    rmse_scores = []
    r2_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate evaluation metrics
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        # Store metrics in lists
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
    
    # Return the average metrics
    return np.mean(mse_scores), np.mean(mae_scores), np.mean(rmse_scores), np.mean(r2_scores)

<h3>K-Nearest Neighbors</h3>

In [12]:
from sklearn.neighbors import KNeighborsRegressor

# Step 1: Initialize your KNN model with the best parameters
knn = KNeighborsRegressor(n_neighbors=3)

# Perform cross-validation on X with amenities
mse_cv, mae_cv, rmse_cv, r2_cv = perform_cross_validation(X, y, knn, kf)
print("With amenities:")
print("Average MSE:", mse_cv)
print("Average MAE:", mae_cv)
print("Average RMSE:", rmse_cv)
print("Average R²:", r2_cv)

With amenities:
Average MSE: 0.000254369059531808
Average MAE: 0.0047532904545668055
Average RMSE: 0.014739945126181653
Average R²: 0.36509065943909214


<h3>Decision Tree</h3>

In [11]:
from sklearn.tree import DecisionTreeRegressor

# Define the Decision Tree Regressor model
model = DecisionTreeRegressor(random_state=42, max_depth=5)

# Perform cross-validation on X with amenities
mse_cv, mae_cv, rmse_cv, r2_cv = perform_cross_validation(X, y, model, kf)
print("With amenities:")
print("Average MSE:", mse_cv)
print("Average MAE:", mae_cv)
print("Average RMSE:", rmse_cv)
print("Average R²:", r2_cv)

With amenities:
Average MSE: 0.00020323232170694873
Average MAE: 0.00513735003673951
Average RMSE: 0.013426026684218679
Average R²: 0.3947073993105753


<h3>Ridge Regression</h3>

In [6]:
from sklearn.linear_model import Lasso, Ridge, BayesianRidge

model = Ridge()
# Perform cross-validation on X with amenities
mse_cv, mae_cv, rmse_cv, r2_cv = perform_cross_validation(X, y, model, kf)
print("With amenities:")
print("Average MSE:", mse_cv)
print("Average MAE:", mae_cv)
print("Average RMSE:", rmse_cv)
print("Average R²:", r2_cv)

With amenities:
Average MSE: 0.00037097892081127266
Average MAE: 0.007903802014210363
Average RMSE: 0.018237966639140617
Average R²: -0.0010789449775610472
With amenities:
Average MSE: 0.0002743748101912808
Average MAE: 0.005004616201473173
Average RMSE: 0.015006467208663027
Average R²: 0.3635129401434203


In [10]:
model = BayesianRidge()

mse_cv, mae_cv, rmse_cv, r2_cv = perform_cross_validation(X, y, model, kf)
print("With amenities:")
print("Average MSE:", mse_cv)
print("Average MAE:", mae_cv)
print("Average RMSE:", rmse_cv)
print("Average R²:", r2_cv)

With amenities:
Average MSE: 0.0002745365647429806
Average MAE: 0.005023555397501788
Average RMSE: 0.015013957463593176
Average R²: 0.36267266395294623


<h3>XGBoost</h3>

In [14]:
import xgboost as xgb

# Initialize the XGBoost model for regression
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6)

# Perform cross-validation on X with amenities
mse_cv, mae_cv, rmse_cv, r2_cv = perform_cross_validation(X, y, model, kf)
print("With amenities:")
print("Average MSE:", mse_cv)
print("Average MAE:", mae_cv)
print("Average RMSE:", rmse_cv)
print("Average R²:", r2_cv)

With amenities:
Average MSE: 0.00017591470458236416
Average MAE: 0.003666813312558292
Average RMSE: 0.012106892724692734
Average R²: 0.5611227962566483


<h3>Random Forest</h3>

In [9]:
from sklearn.ensemble import RandomForestRegressor

# Define the Random Forest model
model = RandomForestRegressor(random_state=42)

# Perform cross-validation on X with amenities
mse_cv, mae_cv, rmse_cv, r2_cv = perform_cross_validation(X, y, model, kf)
print("With amenities:")
print("Average MSE:", mse_cv)
print("Average MAE:", mae_cv)
print("Average RMSE:", rmse_cv)
print("Average R²:", r2_cv)

With amenities:
Average MSE: 0.00015887670398587092
Average MAE: 0.003338814625407941
Average RMSE: 0.011639657054381929
Average R²: 0.5442543241559806


<h3>Feature Importance Analysis</h3>


In [ ]:
#decision_tree = DecisionTreeRegressor().fit(X, y)

# Get feature importances
#feature_importances = decision_tree.feature_importances_

# Create a DataFrame for better visualization
#importance_df = pd.DataFrame({
#    'Feature': X.columns,
#    'Importance': feature_importances
#}).sort_values(by='Importance', ascending=False)

# Display only the top 5 most important features
#top_5_features = importance_df.head(20)

#print(top_5_features)